> Dana jest tablica A długości $ n $. Wartości w tablicy pochodzą ze zbioru $ B $, gdzie $ |B| = log(n) $. Proszę zaproponować możliwie jak najszybszy algorytm sortowania tablicy A.

##### Funkcja testująca poprawność algorytmów

In [1]:
import random

def test_sort(sorting_fn, *, 
              samples=20,         # A number of tests that will be performed
              val_counts=(0, 50), # A minimum and maximum number of values to sort that will be generated
              range_=(-100, 100), # A range which will be used to create a random list of values from this range
              modifies_arr=True,  # Information whether an algorithm modifies the initial array or returns a sorted array
              failed_only=False,  # Show only failed tests. Works only if no_results is set to False
              print_out_fn=None,  # An user-defined function to print additional information. Works only if no_results is set to False
              no_results=False    # When set to True, no results will be printed (useful only for benchmark)
             ): 
    passed = 0                   
    for i in range(samples):
        random_lst = [random.randint(*range_) for _ in range(random.randint(*val_counts))]
        random_lst_before = random_lst[:]
        expected = sorted(random_lst)
        output = sorting_fn(random_lst)
        if not modifies_arr:
            random_lst = output
        is_correct = random_lst == expected
        passed += is_correct
        
        if not no_results:
            if not failed_only or (failed_only and not is_correct):
                print(f'TEST #{i+1}:')
                print(f'Before sorting: {random_lst_before}')
                print(f'After sorting: {random_lst}')
                print(f'Expected result: {expected}')
                print(f'Test {"PASSED" if is_correct else "FAILED"}')
                print(f'Current passed-to-tested ratio: {passed}/{i+1}')
                if print_out_fn:
                    print(f'========== Additional results after sorting  ==========')
                    print_out_fn(random_lst)
                print()
                
    if not no_results:
        print(f'Sorting algorithm is {"correct" if passed == samples else "wrong"}')
        print(f'Passed tests in total: {passed}/{samples}')

# I sposób

### Omówienie algorytmu

Do posortowania tablicy wykorzystamy algorytm Quicker Sort. Ponieważ wiemy, że jest $ log(n) $ różnych wartości, za każdym razem możemy się spodziewać, że wybrany przez nas pivot będzie się powtarzał w tablicy około $ \frac{n}{log(n)} $ razy. Jeżeli w funkcji partition będziemy dzielić tablicę na 3 części, z których pierwsza zawiera elementy mniejsze od pivota, druga mu równe, a trzecia większe od niego, w każdym wywołaniu rekurencyjnym funkcji sortującej będziemy odrzucać $ \frac{n}{log(n)} $ wartości równych pivotowi, które już napewno znajdują się na dobrej pozycji. Wynika stąd, że w najgorszym przypadku będziemy mieli $ log(n) $ zejść rekurencyjnych, ponieważ $ log(n) \cdot \frac{n}{log(n)} = n $ - będzie tak, gdy za każdym razem funkcja partition podzieli tablicę na elementy równe pivotowi, a elemtów większych od pivota (lub mniejszych) nie będzie wcale. Zatem całkowita złożoność wyniesie $ n \cdot log(log(n)) $.

### Implementacja algorytmu

Implementacja algorytmu w tym sposobie jest równoznaczna z zaimplementowaniem funkcji sortującej:

    quicker_sort(arr)

In [2]:
def quicker_sort(arr):
    _quicker_sort(arr, 0, len(arr) - 1)
    

def _quicker_sort(arr, left_idx, right_idx):
    while left_idx < right_idx:
        lt_pivot_last, gt_pivot_first = _partition(arr, left_idx, right_idx)
        
        # If a number of elements lower tha a pivot is greater than a number
        # of elements greater than a pivot, sort recursively the shorter part
        # of elements which are greater than a pivot
        if lt_pivot_last - left_idx > right_idx - gt_pivot_first:
            _quicker_sort(arr, gt_pivot_first, right_idx)
            right_idx = lt_pivot_last  # I removed a tailing recursion
        # Otherwise, sort a subarray of elementslower than a pivot first
        # as it is shorter than a subarray of elements greater than a pivot
        else:
            _quicker_sort(arr, left_idx, lt_pivot_last)
            left_idx = gt_pivot_first  # I removed a tailing recursion

        
def _partition(arr, left_idx, right_idx):
    pivot = arr[left_idx]
    
    # Partition an array into 3 subarrays (lower than, equal to and
    # greater than a pivot value)
    i = left_idx     # A pointer of the first pivot
    j = left_idx + 1 # A pointer of the element after the last pivot
    k = j            # A pointer of the currently checked element 
    while k <= right_idx:
        if arr[k] < pivot:
            _two_swaps(arr, i, j, k)
            i += 1
            j += 1
        elif arr[k] == pivot:
            _one_swap(arr, j, k)
            j += 1
        k += 1
        
    # Return the last index of the subarray of elements lower than a pivot
    # and the first index of the subarray of elements greater than a pivot
    return i - 1, j

    
def _one_swap(arr, i, j):
    # Swap two elements in an array
    arr[i], arr[j] = arr[j], arr[i]
    

def _two_swaps(arr, i, j, k):
    # Rotate right the elements of the indices specified
    arr[k], arr[j], arr[i] = arr[j], arr[i], arr[k]

###### Kilka testów

In [3]:
import math

n = 10_000
limit = math.ceil(math.log(n, 2))
range_ = (-limit, limit)
counts = (n - 1, n)

test_sort(quicker_sort, range_=range_, val_counts=counts, samples=100, failed_only=True)

Sorting algorithm is correct
Passed tests in total: 100/100


# II Sposób

### Omówienie algorytmu

###### I Krok

Najpierw tworzymy pomocniczą tablicę, w której będziemy przechowywać $ log(n) $ unikatowych wartości. Ponieważ całkowita złożoność obliczeniowa algorytmu ma wynieść $ O(n \cdot log(log(n))) $, czynnik $ log(log(n)) $ sugeruje nam, że konieczne będzie użycie wyszukiwania binarnego na tablicy unikatowych wartości. Takie wyszukiwanie działa w czasie $ O(log(k)) $, gdzie $ k $ - liczba elementów przeszukiwanej posortowanej tablicy, ale w naszym przypadku ta tablica ma rozmiar $ k = log(n) $, więc otrzymujemy złożoność jednego binarnego przeszukania tablicy równą $ O(log(log(n))) $. Ponieważ początkowo nie mamy utworzonej tej tablicy, idąc po kolejnych wartościach źródłowej n-elementowej tablicy, musimy sprawdzać, czy bieżąca wartość znajduje się już w tablicy unikatów. Jeżeli nie, konieczne jest jej umieszczenie w tablicy, a następnie przywrócenie niemalejącego porządku wartości w tej tablicy. Najłatwiej jest to uczynić, korzystając ze zmodyfikowanego Insertion Sorta, którego wywołamy tylko dla ostatniej (nowo dodanej) wartości do tablicy. Wynikiem takiej operacji będzie przesunięcie wartości w lewo na odpowiednią pozycję w tablicy, przy czym wszystkie wartości większe od nowo dodanej zostaną przesunięte w prawą stronę. Taka operacja może się wydawać mało efektywna, ale tablica ma maksymalnie wielkość $ log(n) $ elementów, więc wstawianie do tablicy będziemy przeprowadzać jedynie $ log(n) $ razy, więc w najgorszym przypadku (gdy konieczne będzie przesunięcie każdej z dodanych wartości na początek tablicy unikatów) złożoność obliczeniowa wymagana do utworzenia tej tablicy będzie równa $O(log^2(n))$. Warto jeszcze zauważyć, że wstawianie do tablicy zostanie wykonane jedynie, jeżeli obecnie sprawdzana wartość nie występuje w tablicy unikatów, więc finalna złożoność obliczeniowa 1. kroku wyniesie: $ O(n \cdot log(log(n)) + log^2(n)) = O(n \cdot log(log(n))) $.

###### II Krok


Tu na myśl przychodzi od razu algorytm sortowania Counting Sort. Zauważmy jednak, że nie wiemy nic o zakresie liczb, więc w szczególności mogą to być liczby ujemne rzędu $ -10^9 $ lub jeszcze mniejsze i podobnie liczby dodatnie rzędu $ 10^9 $ lub większe. Counting Sort (który nie wykorzystuje Hash Map do przechowywania zliczonych wartości, a na razie nie możemy ich wykorzystywać) wymaga zaalokowania ciągłego obszaru pamięci od wartości najmniejszej do największej włącznie. Może to spowodować zużycie ogromnej ilości pamięci operacyjnej lub jej przepełnienie, dlatego, pamiętając o wymaganej złożoności obliczeniowej, wykorzystamy nieco wolniejsze, ale wciąż wystarczająco szybkie rozwiązanie. Mianowicie, stworzymy nową tablicę o rozmiarze identycznym do wcześniej utworzonej tablicy unikatowych wartości, w której będziemy przechowywać liczby wystąpień kolejnych wartości od najmniejszej do największej. Potrzebna nam jest funkcja, która będzie dla danej wartości zwracała indeks odpowiedniej komórki tablicy, pod którym należy zwiększyć licznik. Możemy zauważyć, że da się wykorzystać do tego celu poprzednio utworzoną tablicę posortowanych unikatów (długości $ log(n) $). Sama funkcja będzie pobierała bieżącą wartość z tablicy wejściowej i z pomocą wyszukiwania binarnego, przeszukiwała posortowaną tablicę unikatów (w czasie $ O(log(log(n)) $), zwracając na koniec indeks danej wartości w tej tablicy unikatów. Następnie pod tym samym indeksem, pod którym znajduje się dana unikatowa wartość, będziemy zwiększać licznik w tablicy counts.


###### III Krok


Teraz pozostała najłatwiejsza część algorytmu, czyli przepisanie wartości w odpowiedniej kolejności. Jest to o tyle łatwe, że mamy tablicę posortowanych wartości i tablicę liczby ich wystąpień, dlatego możemy po prostu liniowo (w czasie $ O(log(n)) $, bo rozmiar tablicy unikatów wynosi $ log(n) $) przeglądać kolejne wartości z tablicy unikatów i umieszczać ich tyle w wejściowej tablicy, ile wynosi licznik odpowiadający danej wartości (ten sam indeks w tablicy liczników, co w tablicy unikatów).


### Implementacja algorytmu

In [4]:
def crazy_sort(arr):
    unique = []
    # Look for unique numbers and store them in a unique array
    for val in arr:
        idx = binary_search(unique, val)
        # If a value is not stored in an unique array, insert this value
        if idx < 0:
            insert_element(unique, val)
    
    counts = [0] * len(unique)
    # Count repetitions of each unique value
    for val in arr:
        idx = binary_search(unique, val)
        counts[idx] += 1
    
    arr_idx = 0
    # Rewrite all values to the inintal array
    for i in range(len(unique)):
        for _ in range(counts[i]):
            arr[arr_idx] = unique[i]
            arr_idx += 1
        
    
def insert_element(arr, val: 'inserted value'):
    arr.append(val)
    if len(arr) > 1:
        # Move all elements that are greater than a value inserted to the right
        idx = len(arr) - 1
        while idx > 0 and arr[idx - 1] > val:
            arr[idx] = arr[idx - 1]
            idx -= 1
        # Place our value on the final position
        arr[idx] = val
        

def binary_search(arr: 'sorted sequence', val: 'searched value') -> int:
    left_idx = 0
    right_idx = len(arr)-1
    
    while left_idx <= right_idx:
        mid_idx = (left_idx + right_idx) // 2
        if val > arr[mid_idx]:
            left_idx = mid_idx + 1
        else:
            right_idx = mid_idx - 1
            
    return left_idx if left_idx < len(arr) and arr[left_idx] == val else -1

###### Kilka testów

In [5]:
import math

n = 10_000
limit = math.ceil(math.log(n, 2))
range_ = (-limit, limit)
counts = (n - 1, n)

test_sort(crazy_sort, range_=range_, val_counts=counts, samples=100, failed_only=True)

Sorting algorithm is correct
Passed tests in total: 100/100
